In [1]:
import pandas as pd

In [2]:
anl_df = pd.read_csv('analysis_data.csv',encoding='utf-16')
du_df = pd.read_csv('duration_survey_df.csv',encoding='utf-16')
df = pd.read_csv('aes_df_pro.csv')

In [3]:
## Data 전처리
du_df = du_df[['project_id','duration']]
du_df.columns = ['project_id','video_duration']

In [4]:
rel_df = pd.merge(anl_df,du_df,on='project_id')

In [5]:
Analysis_df = rel_df[['project_id','duration','project_goal','video_duration','update_count','project_blurb','faq_count',\
    'created_count','backed_count','ratio1','ratio2','ratio3','backer1','backer2','backer3','success']]

In [6]:
# Project_blurb
Analysis_df['project_blurb'] = Analysis_df['project_blurb'].apply(lambda x : len(x))
# Backed_count
Analysis_df['backed_count'] = Analysis_df['backed_count'].apply(lambda x : 0 if x =='None' else x)
Analysis_df['backed_count'] = Analysis_df['backed_count'].apply(lambda x : float(x))

/Users/yonggeol/miniconda3/envs/py/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/yonggeol/miniconda3/envs/py/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/yonggeol/miniconda3/envs/py/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [7]:
analysis_col = ['project_id','BalacingElements', 'ColorHarmony', 'Content', 
    'DoF', 'Light', 'Object', 'Repetition', 'RuleOfThirds', 'Symmetry', 'VividColor','Score']

In [8]:
aes_df = df[analysis_col]

In [9]:
aes_df.head()

,project_id,BalacingElements,ColorHarmony,Content,DoF,Light,Object,Repetition,RuleOfThirds,Symmetry,VividColor,Score
0,669731968,-0.039400,0.078795,-0.168624,-0.008868,-0.117475,0.143944,0.054664,-0.075746,0.027599,-0.217975,0.520444
1,475119411,0.084730,0.347241,0.272903,-0.012265,0.028955,-0.051417,0.042720,0.027142,0.021942,-0.074102,0.613511
2,149769944,-0.004369,0.147379,0.135524,0.009826,-0.088443,0.524562,0.071862,-0.059515,0.047052,-0.189603,0.569721
3,1883635834,-0.023761,0.082842,-0.196100,0.019896,-0.160599,0.041255,0.011802,-0.056203,0.006691,-0.192710,0.521484
4,419752392,-0.008043,0.166219,-0.016462,0.082089,-0.021277,0.137687,0.034483,-0.026740,0.012757,-0.085617,0.556743


In [10]:
success_check_df = anl_df[['project_id','success']]

In [11]:
aesf_df = pd.merge(aes_df,success_check_df,on='project_id')

In [12]:
aesf_df.shape

(965, 13)

In [13]:
aesf_df.to_csv('summaried_project.csv',index=False)

## Data set split

In [ ]:
from sklearn.utils import shuffle
total_df = pd.merge(aes_df,Analysis_df,on='project_id')
total_df = shuffle(total_df)

In [ ]:
from sklearn.preprocessing import StandardScaler
ind = total_df.drop(['project_id','success'],axis=1)
scaler = StandardScaler()
scaler.fit(ind)

X_scaled = scaler.transform(ind)
target = total_df['success']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, target, test_size =0.3, random_state=0)
aes_X_train, aes_X_test = X_train[:,:10], X_test[:,:10]
basic_X_train, basic_X_test = X_train[:,10:], X_test[:,10:]

## Aesthetics feature Model

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

# 하이퍼 파라미터 테스트의 수행 속도를 향상시키기 위해 n_estimators를 100으로 감소
xgb_clf = XGBClassifier(n_estimators=500, random_state=156)

params = {
          'max_depth':[5,7,9],
          'min_child_weight':[1,3,5],
          'colsample_bytree':[0.5,0.75]
}

# 하이퍼 파라미터 테스트의 수행속도를 향상시키기 위해 cv를 지정하지 않음
gridcv = GridSearchCV(xgb_clf, param_grid=params)
gridcv.fit(aes_X_train, y_train, early_stopping_rounds=30, eval_metric='auc',\
          eval_set = [(aes_X_train,y_train),(aes_X_test,y_test)])

print('GridSearchCV 최적 파라미터 : ', gridcv.best_params_)

xgb_roc_score = roc_auc_score(y_test,gridcv.predict_proba(aes_X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

## Aesthetics feature와 선행 연구 feature Merge

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

# 하이퍼 파라미터 테스트의 수행 속도를 향상시키기 위해 n_estimators를 100으로 감소
xgb_clf = XGBClassifier(n_estimators=100, random_state=156)

params = {
          'max_depth':[5,7,9],
          'min_child_weight':[1,3,5],
          'colsample_bytree':[0.5,0.75]
}

# 하이퍼 파라미터 테스트의 수행속도를 향상시키기 위해 cv를 지정하지 않음
gridcv = GridSearchCV(xgb_clf, param_grid=params)
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric='auc',\
          eval_set = [(X_train,y_train),(X_test,y_test)])

print('GridSearchCV 최적 파라미터 : ', gridcv.best_params_)

xgb_roc_score = roc_auc_score(y_test,gridcv.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

## 선행연구 Model

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

# 하이퍼 파라미터 테스트의 수행 속도를 향상시키기 위해 n_estimators를 100으로 감소
xgb_clf = XGBClassifier(n_estimators=100, random_state=156)

params = {
          'max_depth':[5,7,9],
          'min_child_weight':[1,3,5],
          'colsample_bytree':[0.5,0.75]
}

# 하이퍼 파라미터 테스트의 수행속도를 향상시키기 위해 cv를 지정하지 않음
gridcv = GridSearchCV(xgb_clf, param_grid=params)
gridcv.fit(basic_X_train, y_train, early_stopping_rounds=30, eval_metric='auc',\
          eval_set = [(basic_X_train,y_train),(basic_X_test,y_test)])

print('GridSearchCV 최적 파라미터 : ', gridcv.best_params_)

xgb_roc_score = roc_auc_score(y_test,gridcv.predict_proba(basic_X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

## Aesthetics + 선행연구 feature importance

In [ ]:
# n_estimators는 1000으로 증가시키고, learning_rate=0.02로 감소, reg_alpha=0.03으로 추가함.
xgb_clf = XGBClassifier(n_estimators=1000, random_state=156, learning_rate=0.02, max_depth=5,\
                       min_child_weight = 5, colsample_byree=0.5, reg_alpha=0.03)

# 성능 평가 지표를 auc로, 조기 중단 파라미터 값은 200으로 설정하고 학습 수행

# 하이퍼 파라미터 테스트의 수행속도를 향상시키기 위해 cv를 지정하지 않음
xgb_clf.fit(X_train, y_train, early_stopping_rounds=200, eval_metric='auc',\
          eval_set=[(X_train, y_train),(X_test,y_test)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC:{0:.4f}'.format(xgb_roc_score))

In [ ]:
col_names = ind.columns.tolist()
for i,col_name in enumerate(col_names):
    print('f{0} : {1}'.format(i,col_name))

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(1,1,figsize=(10,8))
plot_importance(xgb_clf, ax=ax, max_num_features=20,height=0.4)

## 선행연구 비교

In [ ]:
# n_estimators는 1000으로 증가시키고, learning_rate=0.02로 감소, reg_alpha=0.03으로 추가함.
xgb_clf = XGBClassifier(n_estimators=1000, random_state=156, learning_rate=0.02, max_depth=5,\
                       min_child_weight = 5, colsample_byree=0.5, reg_alpha=0.03)

# 성능 평가 지표를 auc로, 조기 중단 파라미터 값은 200으로 설정하고 학습 수행

# 하이퍼 파라미터 테스트의 수행속도를 향상시키기 위해 cv를 지정하지 않음
xgb_clf.fit(basic_X_train, y_train, early_stopping_rounds=200, eval_metric='auc',\
          eval_set=[(basic_X_train, y_train),(basic_X_test,y_test)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(basic_X_test)[:,1],average='macro')
print('ROC AUC:{0:.4f}'.format(xgb_roc_score))

## Classical feature importance

In [ ]:
import Feature_importance

In [ ]:
classical_df = total_df.drop('project_id',axis=1)

In [ ]:
imporntace_model = Feature_importance.feature_importance(classical_df,'success',scaler='std')

## Chisqure

In [ ]:
imporntace_model.chisquare()

## Random forest

In [ ]:
imporntace_model.random_forest()

## Logistic regression

In [ ]:
imporntace_model.logistic_regression()